In [1]:
#使用ResNet18初始化，使用自定义模型训练

In [2]:
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\qyxx-dannet.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\qyxx-dannet.pkl"  #修改此处文件名

In [5]:
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
from torchvision import models
class DAN(nn.Module):
    def __init__(self, num_class=7,num_head=4, pretrained=True):
        super(DAN, self).__init__()
        
        resnet = models.resnet18(pretrained)
        if pretrained:
            resnet = models.resnet18(pretrained=True)

        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.num_head = num_head
        for i in range(num_head):
            setattr(self,"cat_head%d" %i, CrossAttentionHead())
        self.sig = nn.Sigmoid()
        self.fc = nn.Linear(512, num_class)
        self.bn = nn.BatchNorm1d(num_class)


    def forward(self, x):
        x = self.features(x)
        heads = []
        for i in range(self.num_head):
            heads.append(getattr(self,"cat_head%d" %i)(x))
        
        heads = torch.stack(heads).permute([1,0,2])
        if heads.size(1)>1:
            heads = F.log_softmax(heads,dim=1)
            
        out = self.fc(heads.sum(dim=1))
        out = self.bn(out)
   
        return out, x, heads

class CrossAttentionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = SpatialAttention()
        self.ca = ChannelAttention()
        self.init_weights()


    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
    def forward(self, x):
        sa = self.sa(x)
        ca = self.ca(sa)

        return ca


class SpatialAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1),
            nn.BatchNorm2d(256),
        )
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3,padding=1),
            nn.BatchNorm2d(512),
        )
        self.conv_1x3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(1,3),padding=(0,1)),
            nn.BatchNorm2d(512),
        )
        self.conv_3x1 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3,1),padding=(1,0)),
            nn.BatchNorm2d(512),
        )
        self.relu = nn.ReLU()


    def forward(self, x):
        y = self.conv1x1(x)
        y = self.relu(self.conv_3x3(y) + self.conv_1x3(y) + self.conv_3x1(y))
        y = y.sum(dim=1,keepdim=True) 
        out = x*y
        
        return out 

class ChannelAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.attention = nn.Sequential(
            nn.Linear(512, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.Sigmoid()    
        )


    def forward(self, sa):
        sa = self.gap(sa)
        sa = sa.view(sa.size(0),-1)
        y = self.attention(sa)
        out = sa * y
        
        return out


In [8]:
class AffinityLoss(nn.Module):
    def __init__(self, device, num_class=8, feat_dim=512):
        super(AffinityLoss, self).__init__()
        self.num_class = num_class
        self.feat_dim = feat_dim
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.device = device

        self.centers = nn.Parameter(torch.randn(self.num_class, self.feat_dim).to(device))

    def forward(self, x, labels):
        x = self.gap(x).view(x.size(0), -1)

        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_class) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_class, batch_size).t()
        distmat.addmm_(x, self.centers.t(), beta=1, alpha=-2)

        classes = torch.arange(self.num_class).long().to(self.device)
        labels = labels.unsqueeze(1).expand(batch_size, self.num_class)
        mask = labels.eq(classes.expand(batch_size, self.num_class))

        dist = distmat * mask.float()
        dist = dist / self.centers.var(dim=0).sum()

        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

class PartitionLoss(nn.Module):
    def __init__(self, ):
        super(PartitionLoss, self).__init__()
    
    def forward(self, x):
        num_head = x.size(1)

        if num_head > 1:
            var = x.var(dim=1).mean()
            loss = torch.log(1+num_head/var)
        else:
            loss = 0
            
        return loss

In [9]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [10]:
#to(device)将模型加入GPU中加速计算
model = DAN().to(device)
#设置优化器
criterion_cls = torch.nn.CrossEntropyLoss()
criterion_af = AffinityLoss(device)
criterion_pt = PartitionLoss()

params = list(model.parameters()) + list(criterion_af.parameters())
optimizer = optim.AdamW(params, lr=lr)
#设置损失函数
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
              ReLU-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]          36,864
       BatchNorm2d-6          [128, 64, 56, 56]             128
              ReLU-7          [128, 64, 56, 56]               0
            Conv2d-8          [128, 64, 56, 56]          36,864
       BatchNorm2d-9          [128, 64, 56, 56]             128
             ReLU-10          [128, 64, 56, 56]               0
       BasicBlock-11          [128, 64, 56, 56]               0
           Conv2d-12          [128, 64, 56, 56]          36,864
      BatchNorm2d-13          [128, 64, 56, 56]             128
             ReLU-14          [128, 64,

In [11]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            out,feat,heads = model(x)
            loss = criterion_cls(out,label) + criterion_af(feat,label) + criterion_pt(heads)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(out,dim=1), 1)
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [12]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)
    #内迭代
    for batchidx , (x ,label) in enumerate(data0_train):
#         torch.cuda.empty_cache()
        x , label = x.to(device), label.to(device)
    #在进行梯度更新之前，使用梯度清除，当然当你显存不够的时候，你可以增加判断进行梯度回传，
    #以达到批次增多的效果，如两次64的梯度，如同128的梯度，虽然是可行的，但是我并没有试过。
        optimizer.zero_grad()
        out,feat,heads = model(x)
        loss = criterion_cls(out,label) + 1* criterion_af(feat,label) + 1*criterion_pt(heads)
        _, preds1 = torch.max(F.softmax(out,dim=1), 1)
        #AMP优化
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        optimizer.step()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    val_acc, val_loss = evalute_(model, data1_val)
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
Accuracy : Train is 0.6144736842105263 , Valid is 0.7516983695652174 ;  Loss : Train is  2.5361452805368523 ,Valid is 1.986729570057081
覆盖最好的模型...
epoch: 1
Accuracy : Train is 0.7842105263157895 , Valid is 0.7843070652173912 ;  Loss : Train is  1.7318760282114933 ,Valid is 1.6402023045913032
覆盖最好的模型...
epoch: 2
Accuracy : Train is 0.8332236842105263 , Valid is 0.7982336956521738 ;  Loss : Train is  1.460780039586519 ,Valid is 1.497445821762085
覆盖最好的模型...
epoch: 3
Accuracy : Train is 0.865953947368421 , Valid is 0.8135190217391304 ;  Loss : Train is  1.2634701741369148 ,Valid is 1.33079209016717
覆盖最好的模型...
epoch: 4
Accuracy : Train is 0.8927631578947368 , Valid is 0.8063858695652174 ;  Loss : Train is  1.1073915462744863 ,Valid is 1.2819402269695117
epoch: 5
Accuracy : Train is 0.9138980263157894 , Valid is 0.7860054347826086 ;  Loss : Train is  0.969404641578072 ,Valid is 1.2968656342962515
epoch: 6
Accuracy : Train is 0.9324835526315789 , Valid is 0.8237092391304348 ;  Loss :

Accuracy : Train is 1.0 , Valid is 0.865828804347826 ;  Loss : Train is  0.05082894387213807 ,Valid is 0.6874412920164026
epoch: 58
Accuracy : Train is 1.0 , Valid is 0.8624320652173912 ;  Loss : Train is  0.049787124952203346 ,Valid is 0.7092548816100411
epoch: 59
Accuracy : Train is 1.0 , Valid is 0.8661684782608695 ;  Loss : Train is  0.04974681844836787 ,Valid is 0.692680053088976
epoch: 60
Accuracy : Train is 1.0 , Valid is 0.8637907608695652 ;  Loss : Train is  0.048767069452687314 ,Valid is 0.6974452656248341
epoch: 61
Accuracy : Train is 1.0 , Valid is 0.865828804347826 ;  Loss : Train is  0.04733286609775142 ,Valid is 0.699717280657395
epoch: 62
Accuracy : Train is 1.0 , Valid is 0.8668478260869565 ;  Loss : Train is  0.04756850886501764 ,Valid is 0.6998500823974609
epoch: 63
Accuracy : Train is 1.0 , Valid is 0.8631114130434783 ;  Loss : Train is  0.04727500967289272 ,Valid is 0.6962963316751563
epoch: 64
Accuracy : Train is 1.0 , Valid is 0.865828804347826 ;  Loss : Train is

Accuracy : Train is 1.0 , Valid is 0.8546195652173912 ;  Loss : Train is  0.02235783813031096 ,Valid is 0.7658246846302695
epoch: 119
Accuracy : Train is 1.0 , Valid is 0.8559782608695652 ;  Loss : Train is  0.022321434456266855 ,Valid is 0.7631872166757998
epoch: 120
Accuracy : Train is 1.0 , Valid is 0.8566576086956521 ;  Loss : Train is  0.020895239848055337 ,Valid is 0.7570707435193269
epoch: 121
Accuracy : Train is 1.0 , Valid is 0.858695652173913 ;  Loss : Train is  0.019593743136838863 ,Valid is 0.7413742866205133
epoch: 122
Accuracy : Train is 1.0 , Valid is 0.8617527173913043 ;  Loss : Train is  0.019265132005277432 ,Valid is 0.7372845877771792
epoch: 123
Accuracy : Train is 1.0 , Valid is 0.8583559782608695 ;  Loss : Train is  0.017851311713457108 ,Valid is 0.7491533302742502
epoch: 124
Accuracy : Train is 1.0 , Valid is 0.8603940217391304 ;  Loss : Train is  0.01714264058360928 ,Valid is 0.7529285135476486
epoch: 125
Accuracy : Train is 1.0 , Valid is 0.8583559782608695 ;  L

Accuracy : Train is 0.9991776315789473 , Valid is 0.8512228260869565 ;  Loss : Train is  0.007316161554894949 ,Valid is 0.776932981999024
epoch: 176
Accuracy : Train is 0.9992598684210526 , Valid is 0.850203804347826 ;  Loss : Train is  0.00635186156356021 ,Valid is 0.7561801024105238
epoch: 177
Accuracy : Train is 0.9980263157894737 , Valid is 0.8332201086956521 ;  Loss : Train is  0.010168794638134146 ,Valid is 0.8842927735784779
epoch: 178
Accuracy : Train is 0.997203947368421 , Valid is 0.842391304347826 ;  Loss : Train is  0.013060295348986983 ,Valid is 0.7805635410806407
epoch: 179
Accuracy : Train is 0.9981907894736842 , Valid is 0.8372961956521738 ;  Loss : Train is  0.010288482509847534 ,Valid is 0.8179229355376699
epoch: 180
Accuracy : Train is 0.9985197368421053 , Valid is 0.8447690217391304 ;  Loss : Train is  0.010675971545769197 ,Valid is 0.7833220673644025
epoch: 181
Accuracy : Train is 0.998766447368421 , Valid is 0.8338994565217391 ;  Loss : Train is  0.006997811326168